In [2]:
import sys
!{sys.executable} -m pip install tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer


     |████████████████████████████████| 175.3 MB 40 kB/s s eta 0:00:01  |▍                               | 2.1 MB 2.8 MB/s eta 0:01:02     |███▌                            | 19.2 MB 9.2 MB/s eta 0:00:18     |█████▎                          | 29.1 MB 9.2 MB/s eta 0:00:16     |█████████▋                      | 52.6 MB 32.6 MB/s eta 0:00:04     |█████████████████████▌          | 118.1 MB 46.9 MB/s eta 0:00:02     |████████████████████████████▉   | 158.0 MB 42.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 24.8 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 18.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 32.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 2.8 MB 20.8 MB/s eta 0:00:01
     |████████████████████████████████|

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.0.1/libexec/bin/python3.7 -m pip install --upgrade pip' command.


# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](../images/tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [5]:
# your code here
df = pd.read_csv('../data/tic-tac-toe.csv')
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [6]:
le = LabelEncoder()
le.fit(df.TL)
df.TL = le.transform(df.TL)
df.TM = le.transform(df.TM)
df.TR = le.transform(df.TR)
df.ML = le.transform(df.ML)
df.MM = le.transform(df.MM)
df.MR = le.transform(df.MR)
df.BL = le.transform(df.BL)
df.BM = le.transform(df.BM)
df.BR = le.transform(df.BR)

In [7]:
le.fit(df['class'])
df['class'] = le.transform(df['class'])

X = df.drop(columns='class')
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

norm = Normalizer().fit(X_train)
X_train_scaled = norm.transform(X_train)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `binary_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [18]:
#Initializaing model
model = tf.keras.models.Sequential()

# adding the layers
model.add(tf.keras.layers.Flatten())

# layers with ReLU
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))

# 2 classes
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

# Compiling
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# fitting the training data
model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
29/29 [==============================] - 0s 2ms/step - loss: 0.6377 - accuracy: 0.6538
Epoch 2/5
29/29 [==============================] - 0s 2ms/step - loss: 0.6048 - accuracy: 0.6923
Epoch 3/5
29/29 [==============================] - 0s 3ms/step - loss: 0.5764 - accuracy: 0.7319
Epoch 4/5
29/29 [==============================] - 0s 3ms/step - loss: 0.5525 - accuracy: 0.7308
Epoch 5/5
29/29 [==============================] - 0s 2ms/step - loss: 0.5256 - accuracy: 0.7648


In [19]:
model.save('tic-tac-toe.model')

INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [20]:
X_test_scaled = norm.transform(X_test)

val_loss, val_acc = model.evaluate(X_test_scaled, y_test)

2/2 [==============================] - 0s 1ms/step - loss: 0.5455 - accuracy: 0.7500


In [21]:
# your code here
predictor = tf.keras.models.load_model('tic-tac-toe.model')

for i in range (10):
    idx = np.random.randint(0, len(X_test_scaled))
    print('_______________________________________________')
    print(f"Model prediction probabilities: {predictor.predict(X_test_scaled[idx].reshape(1,-1)).round(3)}")
    print(f"Actual target value: {y_test.iloc[idx]}")
    print('_______________________________________________')

_______________________________________________
Model prediction probabilities: [[0.059 0.941]]
Actual target value: 1
_______________________________________________
_______________________________________________
Model prediction probabilities: [[0.364 0.636]]
Actual target value: 1
_______________________________________________
_______________________________________________
Model prediction probabilities: [[0.188 0.812]]
Actual target value: 0
_______________________________________________
_______________________________________________
Model prediction probabilities: [[0.473 0.527]]
Actual target value: 1
_______________________________________________
_______________________________________________
Model prediction probabilities: [[0.489 0.511]]
Actual target value: 1
_______________________________________________
_______________________________________________
Model prediction probabilities: [[0.436 0.564]]
Actual target value: 0
______________________________________________

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [22]:
# your code here
model = tf.keras.models.Sequential()

# adding the layers
model.add(tf.keras.layers.Flatten())

# Adding 10 layers with ReLU 
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(220, activation=tf.nn.relu))

# 2 classes
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

# Compiling
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# fitting the training data
model.fit(X_train_scaled, y_train, epochs=42)

Epoch 1/42
29/29 [==============================] - 0s 6ms/step - loss: 0.6373 - accuracy: 0.6385
Epoch 2/42
29/29 [==============================] - 0s 6ms/step - loss: 0.6048 - accuracy: 0.6681
Epoch 3/42
29/29 [==============================] - 0s 7ms/step - loss: 0.5648 - accuracy: 0.7154
Epoch 4/42
29/29 [==============================] - 0s 6ms/step - loss: 0.4904 - accuracy: 0.7747
Epoch 5/42
29/29 [==============================] - 0s 6ms/step - loss: 0.5060 - accuracy: 0.7473
Epoch 6/42
29/29 [==============================] - 0s 6ms/step - loss: 0.3986 - accuracy: 0.8319
Epoch 7/42
29/29 [==============================] - 0s 6ms/step - loss: 0.3297 - accuracy: 0.8604
Epoch 8/42
29/29 [==============================] - 0s 6ms/step - loss: 0.3126 - accuracy: 0.8747
Epoch 9/42
29/29 [==============================] - 0s 6ms/step - loss: 0.2656 - accuracy: 0.8857
Epoch 10/42
29/29 [==============================] - 0s 6ms/step - loss: 0.2340 - accuracy: 0.8967
Epoch 11/42
29/29 [

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
''' 
I augmented the layers and the epochs for the model to learn from its errors,
It seems like the most medium layers and epochs the better the model performs.
'''